# Módulo 2
**Desarrollo de proyectos de análisis de datos  IN1002B**

### Detección y corrección de inconsistencias. 



In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

La base de datos **landslide** contiene información de avalanchas ocurridas en todo el mundo.

En esta sesión, exploraremos la base de datos haciendo énfasis sobre que día, mes, año y hora ocurrieron los eventos. Así mismo, realizaremos actividades de limpieza y preparación.

In [ ]:
data = pd.read_csv('landslide.csv')

In [ ]:
# info()


In [ ]:
#shape


In [ ]:
# head()

## **Ajustes previos**

Eliminen las columnas que consideren no relevantes, o bien que no puedan ser sujetas a ajustes de reemplazo:



In [ ]:
data.isnull().sum()

In [ ]:
# Seleccionar las columnas a eliminar (menos la de time, la usaremos despues)
columnas = ['',...]

In [ ]:
# Aplicar la función drop()

data.drop(columns = columnas, inplace = True)
data.info()

## **Fecha y hora**

### Fechas
Cuando queremos trabajar con fechas, podemos usar la funcion de ```to_datetime()```, lo unico que debemos revisar es el orden en el que se cargaron los datos, es decir si se encuentra:
* dd/mm/aaaa
* mm-dd-aaaa
* aaaa dd mm
* etc

En el mismo orden y con los mismos caracteres especiales, le especificaremos a la función que queremos convertir y agregaremos el simbolo de $%$.

* ```%d/%m/%y``` - ```%d/%m/%Y```
* ```%m-%d-%y``` - ```%m-%d-%Y```
* ```%Y %d %m```

Realicemos el ajuste en la columna ```date```

- Impriman un ```head()``` de la columna **date** para ver el formato de la fecha:


In [ ]:
# head()

In [ ]:
data['date'] = pd.to_datetime(data['date'], format= '')

In [ ]:
#info()
#¿qué tipo de dato es nuestra columna date ahora?



### Horas

Ahora realizaremos el ajuste en la columna de ```time```. 


In [ ]:
#unique()


Estos son los valores que requerían un reemplazo.

Los números fueron colocados sin un criterio en especifico, pueden ajustarlos si lo desean.

In [ ]:
data['time'].replace({'Night': '22:00','Early morning': '07:00', 'Afternoon':'13:00','Late morning': '10:00',
                    'Before dawn':'18:00', 'Late afternoon': '16:00', 'evening':'21:00', '****':0 , 'Unknown': 0, 'overnight':'23:00',
                     'Evening':'21:00','Overnight':'01:00', 'Morning':'08:00', ' ':0, 'Late evening': '21:00', 'Late night': '23:00'}, inplace = True)

In [ ]:
# unique()


Como pudieron notar, no existe un formato de **time** en esta ocasión.

Lo bueno es que dentro de ```pd.to_datetime()``` existe una opción llamada **mixed** que infiere el estilo y acomoda la hora.


In [ ]:
data['time'] = pd.to_datetime(data['time'], format='mixed')

In [ ]:
# imrpimir la columna time 
data.time

Ahora, con la función ```isnull()``` veamos cuantos valores faltantes tenemos:

In [ ]:
# isnull()

Considerando esta situación, ¿que tipo de reemplazo consideran apropiado? ¿cuál debería ser la lógica?

Hagamos los pasos necesarios para construir una función. Revisemos cuál es la horá más frecuente:

In [ ]:
data.time.value_counts()

Debido a que no hay una **tendencia** significativa, utilizaremos otra técnica:

### ```groupby()```

Esta función de pandas nos permite agrupar los datos respecto a un conjunto de categorías, donde a su vez les aplicamos una función adicional (media, mediana, moda, etc.)

1. Obtener las modas por país

In [ ]:
#Veamos cuales es la moda respecto a la hora en que ocurrieron los eventos por país

modas = data.groupby('country_name')['time'].agg(
    lambda x: x.mode()[0] if not x.mode().empty else None
)
print(modas)

2. Reemplzar los valores tipo NaT por una de las modas más recurrentes: ```2024-01-01 10:00:00```

In [ ]:
modas = modas.fillna(pd.Timestamp('2024-01-01 10:00:00'))
modas

3. Crear una función de reemplzo 

In [ ]:
def calcular_moda_y_reemplazar_nan(df, col_x, col_y, hora):
    # 1. Agrupar por la columna y calcular la moda de la columna x
    modas = df.groupby(col_y)[col_x].agg(
        lambda x: x.mode()[0] if not x.mode().empty else None
    )
    
    # 2. Reemplazar cualquier NaT que no se haya reemplazado con un valor específico
    df[col_x] = df[col_x].fillna(pd.Timestamp(hora))
    
    # 3. Reemplazar los NaT en el DataFrame original con las modas
    for valor, moda in modas.items():
        df.loc[(df[col_y] == valor) & (df[col_x].isna()), col_x] = moda
    


    return df



df = calcular_moda_y_reemplazar_nan(data, 'time', 'country_name', '2024-09-20 10:00:00')

1) ```df.groupby(col_y)```: Agrupa el DataFrame por la columna ```col_y```, es decir, por los nombres de los países.
2) ```[col_x]```: Selecciona la columna ```col_x``` (donde se encuentran los valores que queremos reemplazar).
3) ```.agg(...)```: Aplica una función de agregación a cada grupo.
 
 ```lambda x: x.mode()[0] if not x.mode().empty else None:```

- Calcula la moda de los valores en cada grupo.
- ```x.mode()``` devuelve una Serie con los valores más frecuentes.

Si hay una moda, toma el primer valor ([0]), y si no hay moda, devuelve None.

-----------------------------------------------------------------------------------------------------------------------------------------------------------

Ahora, impriman un ```value_counts()``` de la columna **time** para ver como quedo:

In [ ]:
# value_counts()

Ahora hagamos un ```dropna()``` en toda la base de datos:

In [ ]:
df.dropna(inplace = True)
df.shape

## **Análisis**

Vamos a extraer la información de las columnas de **date** y **time** para responder algunas preguntas de análisis:

### Més

In [ ]:
df['month'] = df['date'].dt.month

### Año

In [ ]:
df['year'] = df['date'].dt.year

*Las siguientes preguntas aparecerán en su actividad 2.2*

a) ¿En que mes se detectaron más avalanchas? (pueden utilizar: ```.value_counts(normalize = True)```)

b) ¿En que país se detectaron más muertes promedio? (pueden utilizar: ```groupby()[].agg['mean']```).

c) Considerando solo Estados Unidos, ¿en que estado las avalanchas fueron más largas (*distance*) en promedio)?

Pueden consultar el siguiente <a href="https://www.weforum.org/agenda/2016/04/does-this-explain-why-some-landslides-travel-much-further-than-others/">artículo</a> para complementar su respuesta.



In [ ]:
# a)

In [ ]:
# b)

In [ ]:
# c)